# ResNet18 vs ViT-b/16 — Colab Trainer

This notebook mounts Google Drive, clones your GitHub repo, installs dependencies,
links your **data/** folder from Drive, and runs your shell scripts:

- `scripts/run_resnet.sh`
- `scripts/run_vit.sh`
- (optional) `scripts/eval.sh`

All logs and outputs are copied back to Drive.

In [ ]:
# Check GPU (optional)
!nvidia-smi || echo "No GPU detected"

In [ ]:
#@title 🔌 Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
#@title ⚙️ Configuration { run: "auto" }
#@markdown Fill these and run the cells below.

REPO_URL = "https://github.com/Alejandro-Fuste/AI-ResNet-vs-ViT" #@param {type:"string"}
BRANCH = "main"                                                #@param {type:"string"}
PROJECT_NAME = "AI-RESNET-vs-ViT"                              #@param {type:"string"}
DRIVE_DATA = "/content/drive/MyDrive/UCF/Courses/Fall2025/ComputerVisionSystems/HW_1/data.zip"                     #@param {type:"string"}
DRIVE_SAVE = "/content/drive/MyDrive/ai-resnet-vit-outputs"    #@param {type:"string"}

RUN_RESNET = True   #@param {type:"boolean"}
RUN_VIT    = True   #@param {type:"boolean"}
ALSO_EVAL  = True   #@param {type:"boolean"}

In [ ]:
# 🧬 Clone or update the repo
import os, subprocess
workdir = "/content"
repo_dir = os.path.join(workdir, PROJECT_NAME)
if os.path.exists(repo_dir):
    print(f"Repo exists at {repo_dir}. Updating…")
    subprocess.run(["git","fetch","--all"], cwd=repo_dir, check=True)
    subprocess.run(["git","checkout", BRANCH], cwd=repo_dir, check=True)
    subprocess.run(["git","pull","--ff-only"], cwd=repo_dir, check=True)
else:
    subprocess.run(["git","clone","-b", BRANCH, "--single-branch", REPO_URL, PROJECT_NAME], cwd=workdir, check=True)
print("Ready:", repo_dir)

In [ ]:
# 📦 Install requirements
import os, subprocess, sys
req = os.path.join(repo_dir, "requirements.txt")
subprocess.run([sys.executable, "-m", "pip", "install", "--upgrade", "pip", "wheel", "setuptools"], check=True)
if os.path.exists(req):
    subprocess.run([sys.executable, "-m", "pip", "install", "-r", req], check=True)
else:
    print("requirements.txt not found:", req)

In [ ]:
# 📂 Unzip data.zip from Drive
import os, zipfile

zip_path = "/content/drive/MyDrive/UCF/Courses/Fall2025/ComputerVisionSystems/HW_1/data.zip"   # <-- change this to match where your zip is
extract_to = "/content/drive/MyDrive/data"     # this will create the 'data/' folder

os.makedirs(extract_to, exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("Extracted to:", extract_to)

In [ ]:
# 🔗 Link Drive data into the repo as ./data
import os, shutil, time
repo_data = os.path.join(repo_dir, "data")
if os.path.exists(repo_data):
    if os.path.islink(repo_data):
        os.unlink(repo_data)
    else:
        backup = repo_data + "_backup_" + time.strftime("%Y%m%d_%H%M%S")
        print("Backing up existing data folder to", backup)
        shutil.move(repo_data, backup)

if os.path.exists(DRIVE_DATA):
    try:
        os.symlink(DRIVE_DATA, repo_data, target_is_directory=True)
        print("Symlinked", DRIVE_DATA, "->", repo_data)
    except Exception as e:
        print("Symlink failed, copying instead…", e)
        shutil.copytree(DRIVE_DATA, repo_data)
else:
    raise SystemExit(f"Drive data path not found: {DRIVE_DATA}")

In [ ]:
# ✅ Make shell scripts executable
import os, stat
scripts = os.path.join(repo_dir, "scripts")
if os.path.isdir(scripts):
    for f in os.listdir(scripts):
        if f.endswith(".sh"):
            p = os.path.join(scripts, f)
            os.chmod(p, 0o775)
            print("chmod +x", p)
else:
    print("No scripts directory found:", scripts)

## 🧰 Conda setup (Colab) — install Mambaforge

Run this **once per fresh runtime** to install Conda and create the `resnet-vs-vit` environment.

In [ ]:
%%bash
set -euo pipefail

PREFIX=/usr/local
if ! command -v conda >/dev/null 2>&1; then
  echo "[INFO] Installing Mambaforge (Conda)…"
  wget -q https://github.com/conda-forge/miniforge/releases/latest/download/Mambaforge-Linux-x86_64.sh -O /tmp/mamba.sh
  bash /tmp/mamba.sh -b -p $PREFIX/mambaforge
fi

# Enable 'conda' in this shell
source $PREFIX/mambaforge/etc/profile.d/conda.sh

# Create or reuse the env
ENV_NAME=resnet-vs-vit
if ! conda env list | awk '{print $1}' | grep -Fxq "$ENV_NAME"; then
  conda create -y -n "$ENV_NAME" python=3.10
fi

# Show environments
conda env list

## 📦 Install project requirements *into* the Conda env

This ensures your training scripts (which activate `resnet-vs-vit`) see the right packages.

In [ ]:
import os, subprocess

# Use PROJECT_NAME from the config cell
repo_dir = os.path.join("/content", PROJECT_NAME)

cmd = f'source /usr/local/mambaforge/etc/profile.d/conda.sh && conda activate resnet-vs-vit && pip install --upgrade pip wheel setuptools && pip install -r "{repo_dir}/requirements.txt"'
print("[RUN]", cmd)
subprocess.run(["bash","-lc", cmd], check=True)

## 🛠️ Patch training scripts to initialize Conda in non-interactive shells

This makes `bash scripts/run_resnet.sh` and `bash scripts/run_vit.sh` work inside Colab.

In [ ]:
from pathlib import Path
import os, stat

repo_dir = os.path.join("/content", PROJECT_NAME)
scripts_dir = Path(repo_dir) / "scripts"
scripts_dir.mkdir(parents=True, exist_ok=True)

header = r'''#!/usr/bin/env bash
set -euo pipefail

# Make 'conda activate' work in this non-interactive shell
if [ -f "/usr/local/mambaforge/etc/profile.d/conda.sh" ]; then
  . "/usr/local/mambaforge/etc/profile.d/conda.sh"
elif [ -f "/usr/local/etc/profile.d/conda.sh" ]; then
  . "/usr/local/etc/profile.d/conda.sh"
elif command -v conda >/dev/null 2>&1; then
  eval "$(conda shell.bash hook)"
else
  echo "[ERR] Conda not found. Run the Conda setup cell." >&2
  exit 127
fi

conda activate resnet-vs-vit

cd "$(dirname "$0")/.."
mkdir -p logs outputs
'''

resnet_body = r'''
python -m src.train \
  --model resnet18 \
  --pretrained 1 \
  --data_root ./data \
  --img_size 224 \
  --batch_size 64 \
  --epochs 25 \
  --optimizer adamw \
  --lr 3e-4 \
  --weight_decay 1e-4 \
  --seed 42 \
  --outdir ./outputs/resnet18 \
  --log_csv ./logs/resnet18_train.csv
'''.lstrip()

vit_body = r'''
python -m src.train \
  --model vit_b_16 \
  --pretrained 1 \
  --data_root ./data \
  --img_size 224 \
  --batch_size 32 \
  --epochs 25 \
  --optimizer adamw \
  --lr 5e-5 \
  --weight_decay 0.01 \
  --seed 42 \
  --outdir ./outputs/vit_b16 \
  --log_csv ./logs/vit_b16_train.csv
'''.lstrip()

# Write/overwrite scripts
(resnet_path := scripts_dir / "run_resnet.sh").write_text((header + "\n" + resnet_body).strip() + "\n")
(vit_path := scripts_dir / "run_vit.sh").write_text((header + "\n" + vit_body).strip() + "\n")

# Make them executable
os.chmod(resnet_path, 0o775)
os.chmod(vit_path, 0o775)

print("Patched:", resnet_path)
print("Patched:", vit_path)

In [ ]:
# 🚀 Run training scripts with logging to Drive
import os, subprocess, sys, time
from pathlib import Path

os.makedirs(DRIVE_SAVE, exist_ok=True)
timestamp = time.strftime("%Y%m%d_%H%M%S")

def tee_log(cmd, log_path, cwd):
    Path(os.path.dirname(log_path)).mkdir(parents=True, exist_ok=True)
    with open(log_path, "w") as f:
        print("[RUN]", " ".join(cmd))
        p = subprocess.Popen(cmd, cwd=cwd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
        for line in p.stdout:
            sys.stdout.write(line)
            f.write(line)
        p.wait()
        if p.returncode != 0:
            raise SystemExit(f"Command failed with exit code {p.returncode}")

if RUN_RESNET:
    tee_log(["bash","scripts/run_resnet.sh"], os.path.join(DRIVE_SAVE, f"train_resnet18_{timestamp}.log"), cwd=repo_dir)

if RUN_VIT:
    tee_log(["bash","scripts/run_vit.sh"], os.path.join(DRIVE_SAVE, f"train_vit_b16_{timestamp}.log"), cwd=repo_dir)


In [ ]:
# 📊 Run evaluation (eval.sh) separately
import os, time

timestamp = time.strftime("%Y%m%d_%H%M%S")

if os.path.exists(os.path.join(repo_dir, "scripts", "eval.sh")):
    tee_log(["bash","scripts/eval.sh"], os.path.join(DRIVE_SAVE, f"eval_{timestamp}.log"), cwd=repo_dir)
else:
    print("⚠️ eval.sh not found in scripts/")


In [ ]:
# 💾 Copy repo logs/outputs back into Drive for safekeeping
import os, shutil
repo_logs = os.path.join(repo_dir, "logs")
repo_outs = os.path.join(repo_dir, "outputs")
for src, name in [(repo_logs,"logs"), (repo_outs,"outputs")]:
    if not os.path.exists(src):
        print("Skip (missing):", src); continue
    dst = os.path.join(DRIVE_SAVE, name)
    for root, dirs, files in os.walk(src):
        rel = os.path.relpath(root, src)
        tgt_root = os.path.join(dst, rel) if rel != "." else dst
        os.makedirs(tgt_root, exist_ok=True)
        for file in files:
            s = os.path.join(root, file)
            t = os.path.join(tgt_root, file)
            try:
                shutil.copy2(s, t)
            except Exception as e:
                print("Copy warn:", s, "->", t, e)
print("Synced to", DRIVE_SAVE)

In [ ]:
# 🧾 Freeze environment for reproducibility
import subprocess, sys, time, os
ts = time.strftime("%Y%m%d_%H%M%S")
freeze_path = os.path.join(DRIVE_SAVE, f"pip_freeze_{ts}.txt")
with open(freeze_path, "w") as f:
    subprocess.run([sys.executable, "-m", "pip", "freeze"], text=True, stdout=f)
print("Wrote", freeze_path)